### Runtime setup
The following envs enable stable retries and quiet streaming.

- `SCILLM_FORCE_HTTPX_STREAM=1`
- `LITELLM_MAX_RETRIES=3`, `LITELLM_RETRY_AFTER=1`, `LITELLM_TIMEOUT=45`
- Requires `tenacity` installed for backoff.

In [ ]:
import os
os.environ.setdefault('SCILLM_FORCE_HTTPX_STREAM','1')
os.environ.setdefault('LITELLM_MAX_RETRIES','3')
os.environ.setdefault('LITELLM_RETRY_AFTER','1')
os.environ.setdefault('LITELLM_TIMEOUT','45')
try:
    import tenacity  # noqa: F401
    print('tenacity: ok')
except Exception:
    print('tenacity missing — run: pip install tenacity')


# Advanced — Streaming and Tools

## Curl sanity (documentation)

        ```bash
        curl -X POST \
          $CHUTES_API_BASE/chat/completions \
          -H "Authorization: Bearer $CHUTES_API_KEY" \
          -H "Content-Type: application/json" \
          -d '{
            "model": "$CHUTES_VLM_MODEL",
            "messages": [{"role":"user","content":"Tell me a 250 word story."}],
            "stream": true,
            "max_tokens": 256,
            "temperature": 0.7
          }'
        ```

In [ ]:
import os, asyncio
from scillm import acompletion
async def demo_stream():
  stream = await acompletion(
    model=os.environ.get('CHUTES_VLM_MODEL', os.environ['CHUTES_MODEL']),
    api_base=os.environ['CHUTES_API_BASE'],
    api_key=None,
    custom_llm_provider='openai_like',
    extra_headers={'Authorization': f"Bearer {os.environ['CHUTES_API_KEY']}"},
    messages=[{'role':'user','content':'In one word, say OK'}],
    temperature=0,
    max_tokens=8,
    stream=True,
  )
  async for ev in stream:
    d = getattr(ev,'delta',None) or ev
    text = (d.get('content') if isinstance(d,dict) else getattr(d,'content',None)) or ''
    if text: print(text,end='')
  print()
import sys
loop = asyncio.get_event_loop()
try:
  if loop.is_running():
    fut = asyncio.run_coroutine_threadsafe(demo_stream(), loop)
    fut.result()
  else:
    loop.run_until_complete(demo_stream())
except RuntimeError:
  asyncio.run(demo_stream())

In [ ]:
import os, json
from scillm import completion
tools=[{'type':'function','function':{'name':'ack','description':'Acknowledge','parameters':{'type':'object','properties':{'ok':{'type':'boolean'}},'required':['ok']}}}]
resp = completion(
  model=os.environ.get('CHUTES_TOOLS_MODEL', os.environ.get('CHUTES_MODEL_ADVANCED', os.environ['CHUTES_MODEL'])),
  api_base=os.environ['CHUTES_API_BASE'],
  api_key=None,
  custom_llm_provider='openai_like',
  extra_headers={'Authorization': f"Bearer {os.environ['CHUTES_API_KEY']}"},
  messages=[{'role':'user','content':'Call ack with ok=true'}],
  tools=tools,
  temperature=0,
  max_tokens=32,
)
print(getattr(resp.choices[0], 'tool_calls', None))